# Data visualization for dataset hihgD

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import os
import seaborn as sns

from copy import copy
from IPython.display import HTML
from matplotlib.patches import Rectangle, Ellipse
from pprint import pprint
from rich import print
from sys import platform
from time import time

from read_csv import read_track_csv, read_static_info, read_meta_info

print(sns.__version__)
sns.set()
sns.set_style("whitegrid")
sns.set_context("paper")
sns.color_palette("hls", 8)

def print_bl():
    print("\n")

%matplotlib inline
mlp.rcParams['animation.embed_limit'] = 2**128 #Increased animation size to save gifs if necessary


0.12.2

## Extracting frame data from the dataset.

The class SceneData contains the information of a single csv tuple (tracks, tracks metadata and video metadata). It handles data transformation from track information (in a per-vehicle trajectory basis) to parallel information, that is, which vehicles are present in the scene at each frame of the recording. It also handles distinction based on driving direction, so that vehicles in diferent roadways can be identified and field calculation is not impeded by taking into account vehicles driving in the other direction of the freeway.

It also contains a method to display or save a .gif reconstruction of the csv tuple. Beware, this reconstruction takes a long while to process, since the function that plots each frame of the animation is rather slow to execute - for the moment, no more work will be performed on it, but it is still considered to be a WIP.

The class will be updated with whatever additions are considered to extract the potential field that will be used to train the autoencoder further down the pipeline. Each major addition will have a corresponding subsection in this notebook. The goal is, then, to create a class that contains every transformation necessary, so that it can be then uploaded to the DGX server, which will be much faster in processing the information.

Code is self explanatory, except in some special dataframe accesses, which are suitably commented.

In [134]:
tracks = read_track_csv(dataset_location + "01_tracks.csv")
static_info = read_static_info(dataset_location + "01_tracksMeta.csv")
video_info = read_meta_info(dataset_location + "01_recordingMeta.csv")
im_path = dataset_location + "01_highway.png"

class SceneData:
    '''
    A class that represents an entire recording of the highD dataset.

    Attributes:
        dataset_location: a path to the directory in which the dataset is stored.
        dataset_index: index of the recording that is to be addressed
    '''
    def __init__(self, dataset_location = None, dataset_index = None):

        self.dataset_location = dataset_location
        self.df_location = dataset_location + str(dataset_index).zfill(2) + "_tracks.csv"
        self.static_info_location = dataset_location + str(dataset_index).zfill(2) + "_tracksMeta.csv"
        self.video_info_location = dataset_location + str(dataset_index).zfill(2) + "_recordingMeta.csv"
        self.df_location = dataset_location + str(dataset_index).zfill(2) + "_tracks.csv"

        self.df = pd.read_csv(self.df_location)
        self.static_info = read_static_info(self.static_info_location)
        self.video_info = read_meta_info(self.video_info_location)
        self.frame_delay = 1000 / self.video_info["frameRate"]

        self.df_list = self.get_df_list()
        self.total_frame_number = len(self.df_list)
        self.longest_trajectory = 0
        self.df_direction_list = self.get_df_direction_list()
        self.bg_image = None
        self.bg_image_scaling_factor = None
        self.get_background_img(dataset_location + str(dataset_index).zfill(2) + "_highway.png")


    def get_background_img(self, path):
        '''
        Sets the background image for plotting and creating gifs.
            Parameters:
                path (str): path to the png file that contains the background image
            Returns:
                Nothing
        '''
        self.bg_image = plt.imread(path)
        self.bg_image_scaling_factor = np.max((np.max(self.bg_image.shape), self.longest_trajectory)) / np.min((np.max(self.bg_image.shape), self.longest_trajectory))  # Calculates the scaling factor between img size and trajectory length

    def get_distance_between_vehicles(self, t1, t2):
        '''
        Gets the distance between two vehicles.
        Notation: 
            (x, y) - coordinates of the top left corner of the bounding box of the vehicle.
            (w, h) - width and height of the bounding box.

            Parameters:
                t1: Tuple-like in the form (x1, y1, w1, h1).
                t2: Tuple-like in the form (x2, y2, w2, h2)
            Returns:
                dist: distance between vehicles
        '''
        x1, y1, w1, h1 = t1
        x2, y2, w2, h2 = t2
        c1 = np.array([x1 + w1/2, y1 + h1/2])
        c2 = np.array([x2 + w2/2, y2 + h2/2])
        dist = np.linalg.norm(c1-c2)
        return dist
    
    def get_df_list(self):
        '''
        Refactors information from track format to frame format.
            Parameters:
                None, inputs to the method are attributes of the class
            Returns:
                df_list: a list of dataframes - one per frame of recorded data - in which every vehicle in the scene is included.
        '''
        self.df_list = [
            None
        ] * self.df.frame.nunique()  # Preallocate list to improve speed - number of dataframes is equal to that of video frames
        for frame_number in range(1, self.df.frame.nunique() + 1, 1):  # Iterate through every video frame in the dataframe - frame enumeration starts at 1
            self.df_list[frame_number - 1] = self.df[np.in1d(self.df["frame"].values, [frame_number])]  # Select rows of dataframe with df["frame"] == frame_number
        return self.df_list

    def get_df_direction_list(self):
        '''
        Separates vehicles in frames according to the direction they are driving in.
            Parameters:
                None, inputs to the method are attributes of the class
            Returns:
                df_direction_list: a list of tuples - one tuple per every frame, with two elements: eastbound and westbound vehicles dataframes -.
        '''
        eastbound_vehicles = []  # contains the vehicle_id of every vehicle driving East
        westbound_vehicles = []  # contains the vehicle_id of every vehicle driving West
        for vehicle_id in self.static_info.keys():
            self.longest_trajectory = (
                self.static_info[vehicle_id]["traveledDistance"]
                if self.longest_trajectory
                < self.static_info[vehicle_id]["traveledDistance"]
                else self.longest_trajectory
            )  # get longest trajectory in dataset to calculate img scale factor
            if self.static_info[vehicle_id]["drivingDirection"] == 2.0:
                eastbound_vehicles.append(vehicle_id)
            else:
                westbound_vehicles.append(vehicle_id)

        self.df_direction_list = [None] * df.frame.nunique()  # For every frame, save two dataframes - one for eastbound vehicles and one for westbound vehicles
        dummy_east, dummy_west = None, None
        for idx in range(len(self.df_list)):
            dummy_east = self.df_list[idx][
                [ident in eastbound_vehicles for ident in self.df_list[idx].id]
            ]  # eastbound vehicles
            dummy_west = self.df_list[idx][
                [ident in westbound_vehicles for ident in self.df_list[idx].id]
            ]  # westbound vehicles
            self.df_direction_list[idx] = [dummy_east, dummy_west]
        return self.df_direction_list

    def get_vehicle_groups_from_frame(self, frame):
        '''
        Given a frame, it creates a list in which every element is a dataframe with an ego vehicle and all those vehicles that are less closer than a predetermined distance away.
        The length of the list is the number of vehicles that are present in the frame.

        TODO: dynamic distance threshold - set to 50m at the time.
            Parameters:
                Frame: a dataframe containing information of a specific frame - see get_df_list and get_df_direction_list for details
            Returns:
                group_df_list: a list in which every element contains a vehicle group in the dataframe form
        '''
        group_df_list = [None]*len(frame) #for every vehicle in the frame we will save a list of dataframes in which a vehicle group is represented
        drop_index_list = []

        for i ,(index, vehicle_id) in enumerate(frame.iterrows()): #iterate through every vehicle in the frame - returns index of entry and pd.Series with entry
            t1 = (vehicle_id.x, vehicle_id.y, vehicle_id.width, vehicle_id.height) #get position data of ego vehicle
            new_frame = frame.drop([index], axis = 0) #drop ego vehicle to not compare against itself
            drop_index_list.clear() #clear drop index list - it is faster to allocate it outside the loop than to create it in every iteration
            for index_local,vehicle_id_local in new_frame.iterrows(): #compare the ego vehicle against every vehicle in frame
                t2 = (vehicle_id_local.x, vehicle_id_local.y, vehicle_id_local.width, vehicle_id_local.height) #get position data of other vehicles
                if self.get_distance_between_vehicles(t1, t2) > 50: #if other vehicle is more than 50m away
                    drop_index_list.append(index_local) #store vehicles that are further than 50 m away
            new_frame = frame.drop(drop_index_list, axis = 0) #drop vehicles from dataframe
            index_list = list(new_frame.index) #get indices that were not dropped in a list
            index_list.remove(index) #remove the ego vehicle
            index_list.insert(0, index) #insert the ego vehicle in first place
            new_frame = new_frame.loc[index_list] #reorder the dataframe
            group_df_list[i] = new_frame #store filtered dataframe in list of frame groups

        return group_df_list
    
    def get_vehicle_groups(self):
        '''
        Given a list of frames, it creates the vehicle groups associated to them by calling get_vehicle_groups_from_frame.

            Parameters:
                None, inputs to this method are attributes of the class
            Returns:
                vehicle groups: a list in which every element is another list. The elements of the inner list are dataframes in which the information of a vehicle group is stored
        '''
        self.vehicle_groups = [None] * 2 * self.total_frame_number #preallocate for speed
        #get vehicle groups of eastbound vehicles
        for i in range(self.total_frame_number):
            frame = self.df_direction_list[i][0]
            self.vehicle_groups[i] = self.get_vehicle_groups_from_frame(frame)
            frame = self.df_direction_list[i][1]
            self.vehicle_groups[i+self.total_frame_number-1] = self.get_vehicle_groups_from_frame(frame)
        return self.vehicle_groups
    
    def plot_frame(self, frame_number=None):
        '''
        Visualization function. Plots the bounding boxes of every vehicle in a determined frame.

            Parameters:
                Frame number: frame to plot. If none is given, chosen at random
            Returns:
                Nothing.
        '''
        if frame_number is None:
            frame_number = np.random.randint(0, self.total_frame_number)

        eastbound_df, westbound_df = self.df_direction_list[frame_number]
        fig, ax = plt.subplots()
        if self.bg_image is not None:
            ax.imshow(self.bg_image)

        for index, vehicle in eastbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="r",
                facecolor="none",
            )
            ax.add_patch(rect)
        for index, vehicle in westbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor), #x
                    int(vehicle.y * self.bg_image_scaling_factor), #y
                ),
                int(vehicle.width * self.bg_image_scaling_factor), #width
                int(vehicle.height * self.bg_image_scaling_factor), #height
                linewidth=1,
                edgecolor="g",
                facecolor="none",
            )
            ax.add_patch(rect)
        plt.show()

    def plot_frame_anim(self, frame_number, ax):
        '''
        Visualization function. Plots the bounding boxes of every vehicle in a determined frame. Specifically created for animating a .gif

            Parameters:
                Frame number: frame to plot. Provided by the FuncAnimation class in matplotlib.
                ax: axis in which to plot the information. 
            Returns:
                Nothing.
        '''
        ax.clear()
        ax.imshow(self.bg_image)
        eastbound_df, westbound_df = self.df_direction_list[frame_number]
        for index, vehicle in eastbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="r",
                facecolor="none",
            )
            ax.add_patch(rect)
        for index, vehicle in westbound_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="g",
                facecolor="none",
            )
            ax.add_patch(rect)

    def plot_group(self, group_number = None):
        '''
        Visualization function. Plots the bounding boxes of every vehicle in a determined frame.

            Parameters:
                Group number: group to plot. If none is given, chosen at random
            Returns:
                Nothing.
        '''
        if group_number is None:
            group_number = np.random.randint(0, 2*self.total_frame_number)
        group_df = self.vehicle_groups[group_number]

        fig, ax = plt.subplots()
        if self.bg_image is not None:
            ax.imshow(self.bg_image)

        for index, vehicle in group_df.iterrows():
            rect = Rectangle(
                (
                    int(vehicle.x * self.bg_image_scaling_factor),
                    int(vehicle.y * self.bg_image_scaling_factor),
                ),
                int(vehicle.width * self.bg_image_scaling_factor),
                int(vehicle.height * self.bg_image_scaling_factor),
                linewidth=1,
                edgecolor="b",
                facecolor="none",
            )
            ax.add_patch(rect)
        plt.show()

    def create_gif(self, save_animation=False, frames=None, speed_up_factor=1):
        fig, ax = plt.subplots()
        fig.set_tight_layout(True)
        ax.grid(False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(self.bg_image)
        self.anim = animation.FuncAnimation(
            fig=fig,
            func=self.plot_frame_anim,
            fargs=(ax,),
            frames=frames if frames is not None else self.total_frame_number,
            interval=self.frame_delay / speed_up_factor,
            blit=False,
        )
        # HTML(anim.to_html5_video())

        if save_animation:
            writer = animation.PillowWriter(
                fps=scene_data.video_info["frameRate"] * speed_up_factor,
                metadata=dict(artist="Me"),
                bitrate=900,
            )
            anim.save("test.gif", writer=writer)
        # plt.show()
        return self.anim


/home/lmmartinez/Tesis/codigo_tesis/highD/read_csv.py:81: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group_id, rows in grouped:


In [3]:
def show_gif(scene_data, frames=None, speed_up_factor=1):    
    fig, ax = plt.subplots()
    fig.set_tight_layout(True)
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.imshow(scene_data.bg_image)
    anim = animation.FuncAnimation(
        fig=fig,
        func=scene_data.plot_frame_anim,
        fargs=(ax,),
        frames=frames if frames is not None else scene_data.total_frame_number,
        interval=scene_data.frame_delay/speed_up_factor,
        blit=False,
    )
    #HTML(anim.to_html5_video())

    if 0:
        writer = animation.PillowWriter(
            fps=scene_data.video_info["frameRate"],
            metadata=dict(artist="Me"),
            bitrate=900,
        )
        anim.save("test.gif", writer=writer)
    #plt.show()
    return anim

## Class Instance and tests

Test of the implementation of dataframe collection and separation.

In [135]:
if platform == 'darwin':
    dataset_location = "/Users/lmiguelmartinez/Tesis/datasets/highD/data/"
elif:
    dataset_location = "/home/lmmartinez/Tesis/datasets/highD/data/"
dataset_index = 1 #between 01 and 60

# df = pd.read_csv(dataset_location + "01_tracks.csv")
# tracks = read_track_csv(dataset_location + "01_tracks.csv")
# static_info = read_static_info(dataset_location + "01_tracksMeta.csv")
# video_info = read_meta_info(dataset_location + "01_recordingMeta.csv")
# im_path = dataset_location + "01_highway.png"

start = time()
scene_data = SceneData(dataset_location=dataset_location, dataset_index=dataset_index)
end = time()
print("Time elapsed is:", end - start)
#anim = show_gif(scene_data=scene_data, frames = 500, speed_up_factor = 100)
#HTML(anim.to_jshtml())
#show_gif(scene_data=scene_data)

Time elapsed is: 45.50673222541809

Test of the implementation of vehicle group extraction

In [136]:
vehicle_groups = scene_data.get_vehicle_groups()

In [138]:
for group_df in vehicle_groups[-500]: #sanity check
    display(group_df)

,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.00,0,1011,1018,0,1012,0,0,0,2
338533,22041,1011,74.21,9.34,4.65,1.82,-25.37,0.02,0.12,0.03,...,-24.35,1009,1019,1018,1012,1020,0,0,0,2
338905,22041,1012,74.45,13.13,4.04,1.92,-30.43,0.04,-0.58,-0.02,...,-36.63,1018,1020,0,0,0,1009,1011,1019,3
340852,22041,1018,19.71,12.73,4.75,2.12,-36.63,-0.08,-0.04,0.02,...,0.00,0,1012,0,0,0,0,0,1009,3


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
338533,22041,1011,74.21,9.34,4.65,1.82,-25.37,0.02,0.12,0.03,...,-24.35,1009,1019,1018,1012,1020,0,0,0,2
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.00,0,1011,1018,0,1012,0,0,0,2
338905,22041,1012,74.45,13.13,4.04,1.92,-30.43,0.04,-0.58,-0.02,...,-36.63,1018,1020,0,0,0,1009,1011,1019,3


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
338905,22041,1012,74.45,13.13,4.04,1.92,-30.43,0.04,-0.58,-0.02,...,-36.63,1018,1020,0,0,0,1009,1011,1019,3
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.00,0,1011,1018,0,1012,0,0,0,2
338533,22041,1011,74.21,9.34,4.65,1.82,-25.37,0.02,0.12,0.03,...,-24.35,1009,1019,1018,1012,1020,0,0,0,2


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
340852,22041,1018,19.71,12.73,4.75,2.12,-36.63,-0.08,-0.04,0.02,...,0.0,0,1012,0,0,0,0,0,1009,3
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.0,0,1011,1018,0,1012,0,0,0,2


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
341003,22041,1019,245.60,9.10,4.85,2.22,-28.62,0.07,0.09,0.00,...,-25.37,1011,1023,1012,1020,0,0,0,0,2
341342,22041,1020,248.78,13.65,4.65,1.92,-32.24,0.07,0.02,0.02,...,-30.43,1012,0,0,0,0,1011,1019,1023,3


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
341342,22041,1020,248.78,13.65,4.65,1.92,-32.24,0.07,0.02,0.02,...,-30.43,1012,0,0,0,0,1011,1019,1023,3
341003,22041,1019,245.60,9.10,4.85,2.22,-28.62,0.07,0.09,0.00,...,-25.37,1011,1023,1012,1020,0,0,0,0,2


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
342324,22041,1023,387.77,9.6,4.04,1.82,-24.17,0.04,0.11,0.05,...,-28.62,1019,0,1020,0,0,0,0,0,2


## Group indentification

This subsection covers the identification of groups of vehicles. For every vehicle in every frame, separated by direction, a dataframe in which the ego vehicle, as well as every vehicle that is closer than a predetermined threshold is stored. The threshold can be set to be either a hard parameter, or to depend on the ego vehicle speed.

In [27]:
def get_distance_between_vehicles(t1, t2):
    x1, y1, w1, h1 = t1
    x2, y2, w2, h2 = t2
    c1 = np.array([x1 + w1/2, y1 + h1/2])
    c2 = np.array([x2 + w2/2, y2 + h2/2])
    dist = np.linalg.norm(c1-c2)
    return dist

The following cell contains the transformations that are performed in the first frame of the dataset. The information flow is described in the commentaries of the code.

In [131]:
frame = scene_data.df_direction_list[7][0]
group_df_list = [None]*len(frame) #for every vehicle in the frame we will save a list of dataframes in which a vehicle group is represented

drop_index_list = []
index_list = []
for i ,(index, vehicle_id) in enumerate(frame.iterrows()): #iterate through every vehicle in the frame - returns index of entry and pd.Series with entry
    t1 = (vehicle_id.x, vehicle_id.y, vehicle_id.width, vehicle_id.height) #get position data of ego vehicle
    new_frame = frame.drop([index], axis = 0) #drop ego vehicle to not compare against itself
    drop_index_list.clear() #clear drop index list - it is faster to allocate it outside the loop than to create it in every iteration
    index_list.clear()
    for index_local,vehicle_id_local in new_frame.iterrows(): #compare the ego vehicle against every vehicle in frame
        t2 = (vehicle_id_local.x, vehicle_id_local.y, vehicle_id_local.width, vehicle_id_local.height) #get position data of other vehicles
        if get_distance_between_vehicles(t1, t2) > 50: #if other vehicle is more than 50m away
            drop_index_list.append(index_local) #store vehicles that are further than 50 m away
    new_frame = frame.drop(drop_index_list, axis = 0) #drop vehicles from dataframe
    index_list = list(new_frame.index)
    index_list.remove(index)
    index_list.insert(0, index)
    new_frame = new_frame.loc[index_list]
    group_df_list[i] = new_frame #store filtered dataframe in list of frame groups

for group_df in group_df_list: #sanity check
    display(group_df)

,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
7,8,1,373.32,21.69,4.85,2.12,40.94,0.01,0.34,0.0,...,0.0,0,14,0,0,0,0,0,6,5


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
170,8,3,191.79,21.64,3.94,1.92,35.74,-0.01,0.20,-0.01,...,41.56,14,7,0,0,0,10,11,0,5
855,8,7,160.33,22.14,4.75,2.02,32.67,-0.23,0.32,0.04,...,35.74,3,12,0,0,0,11,0,0,5
1470,8,10,207.73,25.67,9.20,2.50,23.26,0.09,0.17,-0.10,...,23.26,6,11,14,0,3,0,0,0,6
1702,8,11,188.30,25.51,4.14,1.92,24.99,0.06,-0.28,-0.06,...,23.26,10,0,14,3,7,0,0,0,6


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
670,8,6,248.19,25.66,11.82,2.50,23.26,-0.01,-0.02,-0.03,...,0.00,0,10,14,0,3,0,0,0,6
1470,8,10,207.73,25.67,9.20,2.50,23.26,0.09,0.17,-0.10,...,23.26,6,11,14,0,3,0,0,0,6
2247,8,14,287.26,21.85,4.95,1.92,41.56,0.08,0.32,0.00,...,40.94,1,3,0,0,0,0,0,6,5


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
855,8,7,160.33,22.14,4.75,2.02,32.67,-0.23,0.32,0.04,...,35.74,3,12,0,0,0,11,0,0,5
170,8,3,191.79,21.64,3.94,1.92,35.74,-0.01,0.20,-0.01,...,41.56,14,7,0,0,0,10,11,0,5
1470,8,10,207.73,25.67,9.20,2.50,23.26,0.09,0.17,-0.10,...,23.26,6,11,14,0,3,0,0,0,6
1702,8,11,188.30,25.51,4.14,1.92,24.99,0.06,-0.28,-0.06,...,23.26,10,0,14,3,7,0,0,0,6


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
1470,8,10,207.73,25.67,9.20,2.50,23.26,0.09,0.17,-0.10,...,23.26,6,11,14,0,3,0,0,0,6
170,8,3,191.79,21.64,3.94,1.92,35.74,-0.01,0.20,-0.01,...,41.56,14,7,0,0,0,10,11,0,5
670,8,6,248.19,25.66,11.82,2.50,23.26,-0.01,-0.02,-0.03,...,0.00,0,10,14,0,3,0,0,0,6
855,8,7,160.33,22.14,4.75,2.02,32.67,-0.23,0.32,0.04,...,35.74,3,12,0,0,0,11,0,0,5
1702,8,11,188.30,25.51,4.14,1.92,24.99,0.06,-0.28,-0.06,...,23.26,10,0,14,3,7,0,0,0,6


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
1702,8,11,188.30,25.51,4.14,1.92,24.99,0.06,-0.28,-0.06,...,23.26,10,0,14,3,7,0,0,0,6
170,8,3,191.79,21.64,3.94,1.92,35.74,-0.01,0.20,-0.01,...,41.56,14,7,0,0,0,10,11,0,5
855,8,7,160.33,22.14,4.75,2.02,32.67,-0.23,0.32,0.04,...,35.74,3,12,0,0,0,11,0,0,5
1470,8,10,207.73,25.67,9.20,2.50,23.26,0.09,0.17,-0.10,...,23.26,6,11,14,0,3,0,0,0,6


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
1937,8,12,22.32,21.91,3.64,1.82,35.8,0.03,0.12,0.01,...,32.67,7,0,0,0,0,11,0,0,5


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
2247,8,14,287.26,21.85,4.95,1.92,41.56,0.08,0.32,0.00,...,40.94,1,3,0,0,0,0,0,6,5
670,8,6,248.19,25.66,11.82,2.50,23.26,-0.01,-0.02,-0.03,...,0.00,0,10,14,0,3,0,0,0,6


Once the code does what is expected, it is transformed into a function. These functions will be coded into the SceneData class as methods.

In [128]:
def get_vehicle_groups_from_frame(frame):
    group_df_list = [None]*len(frame) #for every vehicle in the frame we will save a list of dataframes in which a vehicle group is represented
    drop_index_list = []

    for i ,(index, vehicle_id) in enumerate(frame.iterrows()): #iterate through every vehicle in the frame - returns index of entry and pd.Series with entry
        t1 = (vehicle_id.x, vehicle_id.y, vehicle_id.width, vehicle_id.height) #get position data of ego vehicle
        new_frame = frame.drop([index], axis = 0) #drop ego vehicle to not compare against itself
        drop_index_list.clear() #clear drop index list - it is faster to allocate it outside the loop than to create it in every iteration
        for index_local,vehicle_id_local in new_frame.iterrows(): #compare the ego vehicle against every vehicle in frame
            t2 = (vehicle_id_local.x, vehicle_id_local.y, vehicle_id_local.width, vehicle_id_local.height) #get position data of other vehicles
            if get_distance_between_vehicles(t1, t2) > 50: #if other vehicle is more than 50m away
                drop_index_list.append(index_local) #store vehicles that are further than 50 m away
        new_frame = frame.drop(drop_index_list, axis = 0) #drop vehicles from dataframe
        group_df_list[i] = new_frame #store filtered dataframe in list of frame groups

    return group_df_list

def get_vehicle_groups(scene_data):
    vehicle_groups = [None] * 2 * scene_data.total_frame_number
    #get vehicle groups of eastbound vehicles
    for i in range(scene_data.total_frame_number):
        frame = scene_data.df_direction_list[i][0]
        vehicle_groups[i] = get_vehicle_groups_from_frame(frame)
        frame = scene_data.df_direction_list[i][1]
        vehicle_groups[i+scene_data.total_frame_number-1] = get_vehicle_groups_from_frame(frame)
    return vehicle_groups

In [129]:
vehicle_groups = get_vehicle_groups(scene_data)

In [139]:
for group_df in vehicle_groups[-500]: #sanity check
    display(group_df)

,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.00,0,1011,1018,0,1012,0,0,0,2
338533,22041,1011,74.21,9.34,4.65,1.82,-25.37,0.02,0.12,0.03,...,-24.35,1009,1019,1018,1012,1020,0,0,0,2
338905,22041,1012,74.45,13.13,4.04,1.92,-30.43,0.04,-0.58,-0.02,...,-36.63,1018,1020,0,0,0,1009,1011,1019,3
340852,22041,1018,19.71,12.73,4.75,2.12,-36.63,-0.08,-0.04,0.02,...,0.00,0,1012,0,0,0,0,0,1009,3


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
338533,22041,1011,74.21,9.34,4.65,1.82,-25.37,0.02,0.12,0.03,...,-24.35,1009,1019,1018,1012,1020,0,0,0,2
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.00,0,1011,1018,0,1012,0,0,0,2
338905,22041,1012,74.45,13.13,4.04,1.92,-30.43,0.04,-0.58,-0.02,...,-36.63,1018,1020,0,0,0,1009,1011,1019,3


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
338905,22041,1012,74.45,13.13,4.04,1.92,-30.43,0.04,-0.58,-0.02,...,-36.63,1018,1020,0,0,0,1009,1011,1019,3
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.00,0,1011,1018,0,1012,0,0,0,2
338533,22041,1011,74.21,9.34,4.65,1.82,-25.37,0.02,0.12,0.03,...,-24.35,1009,1019,1018,1012,1020,0,0,0,2


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
340852,22041,1018,19.71,12.73,4.75,2.12,-36.63,-0.08,-0.04,0.02,...,0.0,0,1012,0,0,0,0,0,1009,3
337876,22041,1009,47.76,8.96,4.75,2.12,-24.35,-0.09,0.02,0.04,...,0.0,0,1011,1018,0,1012,0,0,0,2


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
341003,22041,1019,245.60,9.10,4.85,2.22,-28.62,0.07,0.09,0.00,...,-25.37,1011,1023,1012,1020,0,0,0,0,2
341342,22041,1020,248.78,13.65,4.65,1.92,-32.24,0.07,0.02,0.02,...,-30.43,1012,0,0,0,0,1011,1019,1023,3


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
341342,22041,1020,248.78,13.65,4.65,1.92,-32.24,0.07,0.02,0.02,...,-30.43,1012,0,0,0,0,1011,1019,1023,3
341003,22041,1019,245.60,9.10,4.85,2.22,-28.62,0.07,0.09,0.00,...,-25.37,1011,1023,1012,1020,0,0,0,0,2


,frame,id,x,y,width,height,xVelocity,yVelocity,xAcceleration,yAcceleration,...,precedingXVelocity,precedingId,followingId,leftPrecedingId,leftAlongsideId,leftFollowingId,rightPrecedingId,rightAlongsideId,rightFollowingId,laneId
342324,22041,1023,387.77,9.6,4.04,1.82,-24.17,0.04,0.11,0.05,...,-28.62,1019,0,1020,0,0,0,0,0,2


Saving the information is going to prove complicated. Firstly, let us flatten the list of list, so that we have a unique list of dataframes. Then, we can save the dataframes into a csv file.

In [160]:
print(type(vehicle_groups), type(vehicle_groups[0]))
vehicle_groups_flat = []
for frame in vehicle_groups:
    print(type(frame))
    for group in frame:
        vehicle_groups_flat.append(group)

<class 'pandas.core.frame.DataFrame'>
<class 'list'> <class 'list'>

TypeError: 'NoneType' object is not iterable